# Requisitions and Documents

This example shows the Ovation Service Lab (OSL) APIs for sample accessioning and report download. We'll create a simple `Requisition` with one sample. Next, we'll upload supplemental documents for the requistion (e.g. a face sheet, medication list, etc.). Finally, we'll download the complete report(s) for the requisition.

## Setup

In [ ]:
import uuid

from pprint import pprint
from datetime import date

from ovation.session import connect

## Connection

`s` is a `Session` object representing a connection to the Ovation API

In [ ]:
s = connect(input("Email: "), api='https://services-staging.ovation.io')

Many OSL APIs require the Organization `id`.

In [ ]:
organization_id = input('Organization id: ')

## Creating a Requisition

Create a `container` for the sample (in this case, a Tube). The tube `identifier` and `barcode` will be generated by Ovation. You can supply them using the `identifier` and `barcode` attributes of the `container` if needed:

In [ ]:
tube = s.post(s.path('container'), 
              data={'container': {'type': 'Tube'}},
              params={'organization_id': organization_id})

We need to know which project this Requisition belongs to. If you already know the Project Id, you can skip this step. If you need to look up the project by name, use a query:

In [ ]:
project_name = input("Project name: ")

In [ ]:
project = s.get(s.path('project'), 
                params={'q': project_name, # Find project by name
                        'organization_id': organization_id}).projects[0]
pprint(project)

Create a `Requisition` and the `Sample`:

In [ ]:
# See http://lab-services.ovation.io/api/docs#!/requisitions/createRequisition for additional information 
# that can be transmitted with the Requisition including patient demographics, diagnosis, medications, 
# requested test(s)/panel(s) and billing information
requisition_data = {"identifier": str(uuid.uuid4()), # Any unique (within organization) identifier
                    "template": "RNA Requisition", # The requisition template, for the selected project
                    "custom_attributes": {'my-attribute': 1.0}, # Optional; Requisition custom attributes
                    "samples": [
                        {"identifier": str(uuid.uuid4()), # Any unique (within organization) identifier
                         "date_received": date.today().isoformat(),
                         "custom_attributes": {'my-sample-attribute': 1.0}, # Optional; Sample custom attributes
                         "sample_states": [
                                {"container_id": tube.id,
                                "position": "A01"}
                            ]
                        }
                    ]
                  }

req = s.post(s.path('requisition'), 
             data={'requisition': requisition_data},
             params={'organization_id': organization_id,
                    "project_id": project.id})

pprint(req)

## Uploading documents to the Requisition

Once a `Requisition` is created, you can upload Documents to be stored securely with the Requisition. You may use any document tag(s) to which you have `write` permission. The "Supplemental Documents" label is used specifically for documents associated with the requisiton form and supporting materials.

The simplest way to send the document data is as Base64-encoded data withint he POST:

In [ ]:
local_file_path = "example.pdf"

In [ ]:
import base64

with open(local_file_path, "rb") as document_file:
    document_data = base64.b64encode(document_file.read())
    
doc_body = {
          "document": {
            "name": "file1.txt", # Document name
            "tags": [
              {
                "name": "Supplemental Documents" # Special tag for supporting materials
              }
            ],
            "file_data": document_data
          }
        }

doc = s.post(s.path('documents'),
                data=doc_body,
                params={"requisition_id": req.requisition.id} # Supply the Id of the Requisition that will receive the document
            )

## Downloading complete report(s)

Once a Requisition has been processed, you can retrieve the completed clinical report(s) from the Requisition's "Complete Reports" label.

In [ ]:
report_documents = s.get(s.path('document'),
                             params={"requisition_id": req.requisition.id,
                                     "label": "Complete Reports"})

